In [ ]:
import pandas as pd
import numpy as np
df_usda = None

# Load and Combine the Raw Files
try:
    df_inventory = pd.read_csv('./data/raw/honey_bee_inventory.csv')
    df_loss = pd.read_csv('./data/raw/honey_bee_loss.csv')
    df_affected = pd.read_csv('./data/raw/honey_bee_affected_by.csv')  
    df_usda = pd.concat([df_inventory, df_loss, df_affected], ignore_index=True)
    print("Step 1: Raw files loaded and combined successfully.")

except FileNotFoundError:
    print("🔴 ERROR: One or more of the raw USDA CSV files were not found.")
    print("Please make sure the following files are in the same directory as your notebook:")
    print("  - honey_bee_inventory.csv")
    print("  - honey_bee_loss.csv")
    print("  - honey_bee_affected_by.csv")

if df_usda is not None:

    # Clean Columns and Data Type
    columns_to_keep = ['Year', 'Period', 'State', 'Data Item', 'Value']
    df_usda_clean = df_usda[columns_to_keep].copy()

    df_usda_clean['Value'] = df_usda_clean['Value'].str.replace(',', '', regex=False)
    df_usda_clean['Value'] = pd.to_numeric(df_usda_clean['Value'], errors='coerce')
    df_usda_clean.dropna(subset=['Value'], inplace=True)
    df_usda_clean = df_usda_clean[df_usda_clean['State'] != 'OTHER STATES']
    print("Step 2: Data cleaning complete.")


    # Create Metric Column and Pivot
    conditions = [
        df_usda_clean['Data Item'].str.contains('INVENTORY, MEASURED IN COLONIES'),
        df_usda_clean['Data Item'].str.contains('LOSS, DEADOUT, MEASURED IN PCT OF COLONIES'),
        df_usda_clean['Data Item'].str.contains('AFFECTED BY DISEASE'),
        df_usda_clean['Data Item'].str.contains('AFFECTED BY PESTICIDES'),
        df_usda_clean['Data Item'].str.contains('AFFECTED BY PESTS'),
        df_usda_clean['Data Item'].str.contains('AFFECTED BY VARROA MITES'),
        df_usda_clean['Data Item'].str.contains('AFFECTED BY OTHER CAUSES'),
        df_usda_clean['Data Item'].str.contains('AFFECTED BY UNKNOWN CAUSES')
    ]

    choices = [
        'inventory_colonies',
        'loss_pct',
        'stressor_disease_pct',
        'stressor_pesticides_pct',
        'stressor_pests_pct',
        'stressor_varroa_mites_pct',
        'stressor_other_pct',
        'stressor_unknown_pct'
    ]

    df_usda_clean['Metric'] = np.select(conditions, choices, default='other')

    df_usda_final = df_usda_clean.pivot_table(
        index=['Year', 'State', 'Period'],
        columns='Metric',
        values='Value'
    ).reset_index()
    print("Step 3: Data pivoted successfully.")


    # Final Inspection and Save
    df_usda_final.to_csv('cleaned_usda_bee_data.csv', index=False)
    print("\n--- ✅ Preprocessing Complete! ---")
    print("Final pivoted data has been saved to 'cleaned_usda_bee_data.csv'")

    print("\n--- Final USDA Data Info ---")
    df_usda_final.info()

    print("\n--- First 5 Rows of Final Data ---")
    print(df_usda_final.head())

Step 1: Raw files loaded and combined successfully.
Step 2: Data cleaning complete.
Step 3: Data pivoted successfully.

--- ✅ Preprocessing Complete! ---
Final pivoted data has been saved to 'cleaned_usda_bee_data.csv'

--- Final USDA Data Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5097 entries, 0 to 5096
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Year                       5097 non-null   int64  
 1   State                      5097 non-null   object 
 2   Period                     5097 non-null   object 
 3   inventory_colonies         3321 non-null   float64
 4   loss_pct                   1766 non-null   float64
 5   stressor_disease_pct       1533 non-null   float64
 6   stressor_other_pct         1662 non-null   float64
 7   stressor_pesticides_pct    1598 non-null   float64
 8   stressor_pests_pct         1653 non-null   float64
 9   stressor_unknown_pct   